In [74]:
from google import genai
from PIL import Image
import json
import re

In [87]:
image = Image.open('y.png')

In [ ]:
client = genai.Client(api_key='GANTI KUNCI SENDIRI')

In [89]:
prompt = (
    "Transcribe the nutrition facts table in this image and output the data "
    "as a single **JSON object**. Use keys like 'serving-size', 'energy-kcal', 'fat', 'carbohydrates', 'proteins', 'saturated-fat', 'trans-fat', 'sugars', 'added-sugars', 'sodium', 'salt', and 'fiber'. "
    "Do not include any text outside of the JSON object."
    "If the values are not in the image, fill the values with 0"
    "Pay attention to the unit, normalize the unit so it's stated in g (gram) instead of mg (miligram)"
)

In [90]:
result = client.models.generate_content(
    model='gemini-2.5-flash', 
    contents=[prompt, image]
)

In [91]:
data = result.text
print(data)

```json
{
  "serving-size": "31g",
  "energy-kcal": 70,
  "fat": 0.5,
  "carbohydrates": 14,
  "proteins": 1,
  "saturated-fat": 0,
  "trans-fat": 0,
  "sugars": 1,
  "added-sugars": 0,
  "sodium": 0.015,
  "salt": 0,
  "fiber": 2
}
```


In [92]:
cleanedData = data.strip().replace('```json', '').replace('```', '').replace('\n','').strip()
print(cleanedData)

cleanedData = re.sub(r'\s+', ' ', cleanedData).strip()

{  "serving-size": "31g",  "energy-kcal": 70,  "fat": 0.5,  "carbohydrates": 14,  "proteins": 1,  "saturated-fat": 0,  "trans-fat": 0,  "sugars": 1,  "added-sugars": 0,  "sodium": 0.015,  "salt": 0,  "fiber": 2}


In [93]:
tempDict = {}

for i in cleanedData.split(','):
    x = i.split(':')
    col = (x[0].split('"'))[1]
    num = x[1]

    tempDict[col] = num

    print(i)

{ "serving-size": "31g"
 "energy-kcal": 70
 "fat": 0.5
 "carbohydrates": 14
 "proteins": 1
 "saturated-fat": 0
 "trans-fat": 0
 "sugars": 1
 "added-sugars": 0
 "sodium": 0.015
 "salt": 0
 "fiber": 2}


In [94]:
resDict = {}

for i in tempDict:
    if i == 'serving-size':
        divider = tempDict[i].split('"')
        divider = divider[1].split('g')
        divider = int(divider[0].strip())
    else:
        try:
            resDict[i+'_1g'] =  round(float(tempDict[i].strip())/divider, 3)
        except ValueError:
            value = tempDict[i].split("}")
            value = float(value[0].strip())

            resDict[i+'_1g'] =  round(value/divider, 3)

resDict

{'energy-kcal_1g': 2.258,
 'fat_1g': 0.016,
 'carbohydrates_1g': 0.452,
 'proteins_1g': 0.032,
 'saturated-fat_1g': 0.0,
 'trans-fat_1g': 0.0,
 'sugars_1g': 0.032,
 'added-sugars_1g': 0.0,
 'sodium_1g': 0.0,
 'salt_1g': 0.0,
 'fiber_1g': 0.065}